# 12wk-1: 순환신경망 (2) – RNN

최규빈  
2024-05-15

## E. 구현3 – RNN

ref: <https://pytorch.org/docs/stable/generated/torch.nn.RNN.html>

`-` 데이터정리

In [153]:
x = torch.tensor(df_train.x.map({'A':0,'b':1,'c':2,'d':3}))
y = torch.tensor(df_train.y.map({'A':0,'b':1,'c':2,'d':3}))
X = torch.nn.functional.one_hot(x).float()
y = torch.nn.functional.one_hot(y).float()

`-` Net설계 및 가중치 설정 (구현1,2와 동일하도록 가중치 초기화)

In [154]:
torch.manual_seed(43052)
_rnncell = rNNCell()
cook = torch.nn.Linear(2,4)

In [155]:
rnn = torch.nn.RNN(4,2)
rnn.weight_ih_l0.data = _rnncell.i2h.weight.data 
rnn.weight_hh_l0.data = _rnncell.h2h.weight.data 
rnn.bias_ih_l0.data = _rnncell.i2h.bias.data 
rnn.bias_hh_l0.data = _rnncell.h2h.bias.data 

`-` 손실함수 및 옵티마이저 설정

In [156]:
loss_fn = torch.nn.CrossEntropyLoss() 
optimizr = torch.optim.Adam(list(rnn.parameters())+list(cook.parameters()),lr=0.1)

`-` 학습

In [157]:
Water = torch.zeros(1,2) # 여기에서 1=D*num_layers임 //  D는 단방향=1 양방향=2, num_layers는 stacking RNN의 중첩수 
for epoc in range(200):
    ## 1 
    h,hL = rnn(X,Water)
    netout = cook(h) 
    ## 2 
    loss = loss_fn(netout,y)
    ## 3 
    loss.backward()
    ## 4 
    optimizr.step()
    optimizr.zero_grad()

`-` 결과확인

In [158]:
h,_ = rnn(X,Water)
netout = cook(h)
yhat = soft(netout)
yhat

-   구현1, 구현2와 같은결과